In [1]:
import pandas as pd
import numpy as np
import os
import requests
import json


In [2]:
def data_import():
    """
    Import der Daten aus allen Dateien, die mit 'waqi-covid-' anfangen.
    Entfernen von Kommentaren, Duplikaten und Umbenennung bestimmter Spaltenwerte.
    Zusammenführung der DataFrames.
    """
    data_folder = './data/'
    all_files = [f for f in os.listdir(data_folder) if f.startswith('waqi-covid-') and f.endswith('.csv') or f == 'airquality-covid19-cities.json']
    dataframes = []

    if not all_files:
        print("Keine Dateien gefunden.")
        return None

    for file in all_files:
        file_path = os.path.join(data_folder, file)
        try:
            df = pd.read_csv(file_path, comment='#')

            if "Specie" not in df.columns:
                print(f"Spalte 'Specie' fehlt in {file}")
                continue

            df["Specie"] = df["Specie"].replace("wind gust", "wind-gust")
            df["Specie"] = df["Specie"].replace("wind speed", "wind-speed")
            df = df.drop_duplicates()

            if df.empty:
                print(f"{file} enthält nach Duplikat-Entfernung keine Daten mehr.")
                continue

            dataframes.append(df)
        except Exception as e:
            print(f"Fehler beim Verarbeiten von {file}: {e}")

    if not dataframes:
        print("Keine gültigen Daten vorhanden.")
        return None

    return pd.concat(dataframes, ignore_index=True)

In [3]:
df=data_import()
print(df)

Spalte 'Specie' fehlt in airquality-covid19-cities.json
waqi-covid-2022Q1.csv enthält nach Duplikat-Entfernung keine Daten mehr.
                Date Country    City Specie  count   min    max  median  \
0         2015-01-06      KR  Jeonju     co    124   0.1   12.3     4.5   
1         2015-01-22      KR  Jeonju     co    116   4.5   10.0     6.7   
2         2015-03-30      KR  Jeonju     co    118   1.2   11.2     5.6   
3         2015-05-27      KR  Jeonju     co     93   2.3    5.6     3.4   
4         2015-02-03      KR  Jeonju     co    133   4.5   13.4     7.8   
...              ...     ...     ...    ...    ...   ...    ...     ...   
14251935  2024-08-12      IR  Tehran    so2    154   6.0  101.0    12.0   
14251936  2025-01-13      IR  Tehran    so2    516  10.0   74.0    19.0   
14251937  2024-02-10      IR  Tehran    so2    299   5.0   76.0    34.0   
14251938  2024-05-16      IR  Tehran    so2    559   6.0   92.0    17.0   
14251939  2025-01-04      IR  Tehran    so2   

In [4]:
def geodata(df):
    """
    Fügt die Geodaten zu den Städten hinzu
    """
    df = df.copy()

    # JSON-Datei laden
    with open('./data/airquality-covid19-cities.json', 'r', encoding='utf-8') as file:
        geodata = json.load(file)
    
    geodata = geodata["data"] 

    # Erstellen eines DataFrames mit Städten und Geokoordinaten
    df_places = pd.DataFrame([
        {
            "City": entry["Place"]["name"],  # Stadtname
            "Latitude": entry["Place"]["geo"][0],  
            "Longitude": entry["Place"]["geo"][1]
        }
        for entry in geodata if "Place" in entry])  # Nur Einträge mit "Place" verwenden
    

    # Standardisiere den Stadtnamen für eine bessere Übereinstimmung
    df["City"] = df["City"].str.lower().str.strip()
    df_places["City"] = df_places["City"].str.lower().str.strip()

    # Zusammenführen der beiden DataFrames über "City"
    df = df.merge(df_places, on="City", how="left")

    return df

In [5]:
def data_cleaning(df):
    """Bereinigung der Daten
    - nicht benötigte Spalten löschen
    - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
    - Spalte Species aufteilen
    - df als csv speichern im Datenverzeichnis
    """
    df = df.copy()

    df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

    df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

    df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

    df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')

    output_path = './data/cleaned_data.csv'
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df.to_csv(output_path, index=False)
    print(f"✅ Datei wurde gespeichert: {output_path}")

    #evt hier nochmal umarbeiten
    #geo_data = add_geodata(df)
    #und dann nochmal mergen außerhakb von geodata
    #df = pd.merge(df, geo_data, on=["City"], how="left")
    df = geodata(df)

    # weather_data = add_weather(df)

    # Zusammenführen der beiden DataFrames über "City und Date"
    # df = pd.merge(df, weather_data, on=["City", 'Date'],how="left")

    df= convert_date(df)

    return df

    



In [ ]:
def convert_date(df):
    """
    Teilt die Spalte Date in Year, Month, Day auf
    """
    # Convert 'Date' column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Split 'Date' column into 'year', 'month' and 'day'
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['day'] = df['Date'].dt.day

    # Remove 'Date' column
    if 'Date' in df.columns:
        df.drop(columns=['Date'], inplace=True)

In [6]:
df.head().T

,0,1,2,3,4
Date,2015-01-06,2015-01-22,2015-03-30,2015-05-27,2015-02-03
Country,KR,KR,KR,KR,KR
City,Jeonju,Jeonju,Jeonju,Jeonju,Jeonju
Specie,co,co,co,co,co
count,124,116,118,93,133
min,0.1,4.5,1.2,2.3,4.5
max,12.3,10.0,11.2,5.6,13.4
median,4.5,6.7,5.6,3.4,7.8
variance,55.74,16.09,35.98,6.54,39.24


In [ ]:
# def data_cleaning(df):
#     """Bereinigung der Daten
#     - nicht benötigte Spalten löschen
#     - Eine Stadt pro Land mit den meisten Messwerten 
#     - und die Liste als csv ins Datenverzeichnis speichern
#     - Zusammenfassung der Daten nach Datum, Land, Stadt und Spezies, so dass nur ein Messwert je Species (Median) pro Tag/ Stadt verbleibt
#     - filtern des df nach den ausgewählten Städten
#     - Spalte Species aufteilen
#     - df als csv speichern im Datenverzeichnis
#     """
#     df = df.copy()

#     df = df.drop(columns=['variance', 'min', 'max'], errors='ignore')

#     # city_counts = df.groupby(["Country", "City"]).size().reset_index(name="count")
#     # cities = city_counts.loc[city_counts.groupby("Country")["count"].idxmax()]
  
#     # output_path = './data/city_per_country.csv'
#     # os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     # cities.to_csv(output_path, index=False)
#     # print(f"✅ Datei wurde gespeichert: {output_path}")
    
#     # cities=cities['City'].tolist()
#     # df = df[df['City'].isin(cities)]

#     df = df.groupby(["Date", "Country", "City", "Specie"], as_index=False).agg({"median": "mean"})  

#     df = df.pivot(index=["Date", "Country", "City"], columns="Specie", values='median').reset_index()

#     output_path = './data/cleaned_data.csv'
#     os.makedirs(os.path.dirname(output_path), exist_ok=True)
#     df.to_csv(output_path, index=False)
#     print(f"✅ Datei wurde gespeichert: {output_path}")

#     return df



In [ ]:
df.head().T

In [ ]:
df_, weather_data = data_cleaning(df)


In [ ]:
df_.head().T
weather_data.head().T

In [ ]:
pd.merge(df_, weather_data, on=["City", 'Date'],how="left")

In [ ]:
print(df.head())

In [ ]:
# df=data_cleaning(df)
# print(df.shape)
# df.head()  

In [ ]:
# cities = df[['City', 'Latitude', 'Longitude']]
# print(cities.head())

In [ ]:
# from meteostat import Stations

# # Test mit einer Stadt
# city = {'City': 'Berlin', 'Latitude': 52.5200, 'Longitude': 13.4050}

# # Nächste Wetterstation suchen
# stations = Stations().nearby(city['Latitude'], city['Longitude'])
# station = stations.fetch(5)  # Hol dir mal die nächsten 5 Stationen zum Vergleich

# print(station)


In [ ]:
# from meteostat import Daily
# from datetime import datetime

# # Test mit der ersten Station
# start = datetime(2023, 1, 1)
# end = datetime(2023, 1, 31)

# if not station.empty:
#     station_id = station.index[0]
#     print(f"📡 Verwende Station: {station_id}")

#     # Tägliche Wetterdaten abrufen
#     data = Daily(station_id, start, end)
#     data = data.fetch()

#     print(data.head())
# else:
#     print("⚠️ Keine Wetterstation gefunden!")

In [ ]:
#WEtterdaten integrieren

from meteostat import Daily, Stations
from datetime import datetime

# Städte
cities = df[['City', 'Latitude', 'Longitude']].drop_duplicates()

# Zeitspanne festlegen
start = datetime(2015, 1, 1)
end = datetime(2024, 12, 31)

# DataFrame für alle Städte vorbereiten
all_data = pd.DataFrame()

# Daten für jede Stadt abrufen und hinzufügen
for _, city in cities.iterrows():
    # Nächste Wetterstation suchen
    stations = Stations().nearby(city['Latitude'], city['Longitude'])
    station = stations.fetch(1)

    if not station.empty:
        station_id = station.index[0]

        # Tägliche Wetterdaten abrufen
        data = Daily(station_id, start, end).fetch()

        # Nan-Daten rausfiltern
        data.dropna(how='all', inplace=True)

        if not data.empty:
            # Stadtname hinzufügen
            data["City"] = city["City"]

            # Daten in den Gesamtdaten-Frame einfügen
            all_data = pd.concat([all_data, data])

    # if station.empty:
    #     print(f"⚠️ Keine Wetterstation für {city['City']} gefunden, übersprungen.")
    #     continue

    # # Tägliche Wetterdaten abrufen
    # data = Daily(station.index[0], start, end)
    # data = data.fetch()

    # if data.empty:
    #     print(f"⚠️ Keine Wetterdaten für {city['City']} gefunden, übersprungen.")
    #     continue

    # # Stadtname hinzufügen
    # data["City"] = city["City"]

    # # Daten in den Gesamtdaten-Frame einfügen
    # all_data = pd.concat([all_data, data])

# Index zurücksetzen
all_data.reset_index(inplace=True)

# Spalte time umbennen in Date
all_data.rename(columns={'time': 'Date'}, inplace=True) 

# Standardisiere den Stadtnamen für eine bessere Übereinstimmung
all_data["City"] = all_data["City"].str.lower().str.strip()

print(f"✅ Wetterdaten gesammelt für {all_data['City'].nunique()} Städte")

# # Ergebnis anzeigen
print(all_data.head())

In [ ]:
from meteostat import Daily, Stations
from datetime import datetime

def add_weather(df):
    """Fügt die Wetterdaten zu den Städten hinzu
    """
    # Städte
    cities = df[['City', 'Latitude', 'Longitude']].drop_duplicates()

    # Zeitspanne festlegen
    start = datetime(2015, 1, 1)
    end = datetime(2024, 12, 31)

    # DataFrame für alle Städte vorbereiten
    all_data = pd.DataFrame()

    # Daten für jede Stadt abrufen und hinzufügen
    for _, city in cities.iterrows():
        # Nächste Wetterstation suchen
        stations = Stations().nearby(city['Latitude'], city['Longitude'])
        station = stations.fetch(1)

        if not station.empty:
            station_id = station.index[0]

            # Tägliche Wetterdaten abrufen
            data = Daily(station_id, start, end).fetch()

            # Nan-Daten rausfiltern
            data.dropna(how='all', inplace=True)

            if not data.empty:
                # Stadtname hinzufügen
                data["City"] = city["City"]

                # Daten in den Gesamtdaten-Frame einfügen
                all_data = pd.concat([all_data, data])

    # Index zurücksetzen
    all_data.reset_index(inplace=True)

    # Spalte time umbennen in Date
    all_data.rename(columns={'time': 'Date'}, inplace=True) 

    # Standardisiere den Stadtnamen für eine bessere Übereinstimmung
    all_data["City"] = all_data["City"].str.lower().str.strip()

    # Datum formatieren für bessere Übereinstimmung
    all_data['Date'] = pd.to_datetime(all_data['Date']).dt.strftime('%Y-%m-%d')

    # Berechne den Anteil der NaN-Werte pro Spalte
    missing_percentage = all_data.isna().mean() * 100
    # Lösche die Spalten, bei denen der Anteil an NaN-Werten größer als 80% ist
    all_data = all_data.loc[:, missing_percentage <= 80]
    
    print(f"✅ Wetterdaten gesammelt für {all_data['City'].nunique()} Städte")    

    return all_data

In [ ]:
weather = add_weather(df)
print(weather.head())

In [ ]:
# Zusammenführen der beiden DataFrames über "City und Date"
df = pd.merge(df, all_data, on=["City", 'Date'],how="left")

In [ ]:
df = add_weather(df)
print(df.head())

In [ ]:
# Convert 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# Split 'Date' column into 'year', 'month' and 'day'
df['year'] = df['Date'].dt.year
df['month'] = df['Date'].dt.month
df['day'] = df['Date'].dt.day

# Remove 'Date' column
if 'Date' in df.columns:
    df.drop(columns=['Date'], inplace=True)

In [ ]:
df.sample(5).T

In [ ]:
# #print(df[['City', 'Date']].head())
# print(all_data.head())

In [ ]:
# print(df['Date'].dtype)
# print(all_data['Date'].dtype)
# print(df['City'].dtype)
# print(all_data['City'].dtype)

In [ ]:
# print(df['City'].unique())
# print(all_data['City'].unique())

# print(set(df['Date']).difference(set(all_data['Date'])))
# print(set(df['City']).difference(set(all_data['City'])))

In [ ]:

# print(type(df))
# print(type(all_data))

# print(df.shape)
# print(all_data.shape)


# Konvertiere die 'Date' Spalte in beiden DataFrames zu datetime
df['Date'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m-%d')
all_data['Date'] = pd.to_datetime(all_data['Date']).dt.strftime('%Y-%m-%d')

#print(len(set(df['Date']).difference(set(all_data['Date']))))

In [ ]:
print(all_data.sample(15))


In [ ]:
# # Funktion für Übersicht über dtypes, missing values, unique values etc.
# def overview(df):
#     '''
#     Erstelle einen Überblick über einige Eigenschaften der Spalten eines DataFrames.
#     VARs
#         df: Der zu betrachtende DataFrame
#     RETURNS:
#         None
#     '''
#     display(pd.DataFrame({'dtype': df.dtypes,
#                           'total': df.count(),
#                           'missing': df.isna().sum(),
#                           'missing%': df.isna().mean()*100,
#                           'n_uniques': df.nunique(),
#                           'uniques%': df.nunique()/df.shape[0]*100,
#                           'uniques': [df[col].unique() for col in df.columns]
#                          }))
# overview(all_data)

In [ ]:
#all_data.columns

In [ ]:
# Berechne den Anteil der NaN-Werte pro Spalte
missing_percentage = all_data.isna().mean() * 100

# Lösche die Spalten, bei denen der Anteil an NaN-Werten größer als 80% ist
all_data = all_data.loc[:, missing_percentage <= 80]

In [ ]:
#all_data.columns

In [ ]:


# Zusammenführen der beiden DataFrames über "City"
df = pd.merge(df, all_data, on=["City", 'Date'],how="left")
print(df.shape)
df.sample(30)

In [ ]:
df.columns

In [ ]:
print(df.sample(5).T)
#df.isna().sum()